In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
import qiskit_nature
qiskit_nature.__version__

'0.7.2'

In [3]:
BL = 0.6

In [4]:
# Parameters
BL = 0.2


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [5]:
def get_qubit_op(BL):
    molecule = MoleculeInfo(
        symbols=["H", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    # Get the fermionic Hamiltonian (first element)
    fermionic_op, _ = problem.second_q_ops()

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    # Use Jordan-Wigner (no tapering)
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(fermionic_op)
    # converter = QubitConverter(mapper=mapper)
    # qubit_op = converter.convert(fermionic_op, num_particles=num_particles)

    core_electron_energy = problem.hamiltonian.constants["FreezeCoreTransformer"]
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy

    return (
        qubit_op,
        num_particles,
        num_spatial_orbitals,
        problem,
        mapper,
        nuclear_repulsion_energy,
        core_electron_energy,
    )


In [6]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)

In [7]:
qubit_op

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.33562923+0.j,  0.28220597+0.j, -0.64828256+0.j,  0.14936431+0.j,
  0.28220597+0.j,  0.1910155 +0.j, -0.64828256+0.j,  0.18885749+0.j,
  0.03949318+0.j,  0.03949318+0.j,  0.03949318+0.j,  0.03949318+0.j,
  0.18885749+0.j,  0.20032334+0.j,  0.14936431+0.j])

In [8]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [9]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [10]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [11]:
# l=12 and maxiter = 200 works for LiH and HF

In [12]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [13]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()

In [14]:
vqe_result

{'x': [1.7091456095014528,
  1.9086913745460135,
  2.783866927363235,
  -0.9472617769818815,
  -1.7441133514370606,
  -2.201090978644254,
  0.312208337897507,
  0.33675321475354686,
  -1.408847023775857,
  -0.8322077257394802,
  -0.7627391697058924,
  2.277091206120087,
  1.0340516842435254,
  2.05535394485868,
  0.27200990264308056,
  1.5513114703556832,
  0.15912934812633414,
  0.5416882128827989,
  -0.49160618030114334,
  2.362547035067077,
  2.612356932949136,
  -2.6044039460650494,
  0.5399223665791544,
  0.61369767053536,
  -1.2883942165018727,
  1.9717421728845872,
  -2.424954795123411,
  0.08756232702343639,
  1.0041477009405948,
  -0.13727037356559865,
  -2.147022731671269,
  -0.7959071354235229,
  -2.7431406192073307,
  2.2748591686001864,
  -2.9915005071169904,
  -2.541700434462566,
  -2.4135622822128195,
  -0.17650168774466315,
  1.5061314437516633,
  0.5363493373107474,
  -1.668502414768305,
  0.48449383273410973,
  2.281807605573507,
  1.8569877104699999,
  2.569260377790

In [15]:
len(vqe_result['x'])

104

In [16]:
print(vqe_result['fun'] - vqe_result['dd'])

1.9478703627839877e-07


In [17]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)